# Plato Tutorial

Plato is a package made with two objectives.

1) Simplify the use of Theano.  
2) Provide a good libary of standard Deep Learning algorithms.

This tutorial demonstrates point 1.

## 1. Symbolic functions.

In Plato, we have the concept of *symbolic functions*, which are function that take and return theano symbolic variables.  These functions can be compiled to *numeric functions* which take and return numpy arrays and python ints/floats.

In [6]:
from plato.core import symbolic

@symbolic
def add_two_numbers(x, y):
    return x+y

f = add_two_numbers.compile()
print '3+4=%s' % f(3, 4)

3+4=7


## 2. Adding State

We are also able to create stateful functions.  In the below example, Plato recognises that the return value is in the format `output, [(shared_variable, shared_variable_update)]`, so it creates a stateful function when it compiles.

In [12]:
import theano
from plato.core import symbolic

@symbolic
def counter(start_value = 1):
    counter = theano.shared(start_value)
    return counter, [(counter, counter+1)]

f = counter.compile()
print 'I can count to ten.  See: %s' % ([int(f()) for _ in xrange(10)])

# Note that we start from scratch when we compile the function a new time.
f2 = counter.compile()
print 'I can too: %s' % ([int(f2()) for _ in xrange(10)])


I can count to ten.  See: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
I can too: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


# 3. Classes: Multiple functions sharing a variable.

We often have situations where we have a variable that is shared between two functions (e.g. in a classifier, the weights may be modified by the *train* function and updated by the *predict* function).  We usually deal with this using classes.  As a simple example, lets take the [Collatz Conjecture](https://en.wikipedia.org/wiki/Collatz_conjecture).  

In [28]:
class Collatz:

    def __init__(self, initial_value):
        self.value = theano.shared(initial_value)
        
    @symbolic
    def divide_by_2(self):
        new_value = self.value/2
        return new_value, [(self.value, new_value)]
    
    @symbolic
    def multiply_by_3_and_add_one(self):
        new_value = self.value*3+1
        return new_value, [(self.value, new_value)]
    
c = Collatz(20)
div_fcn = c.divide_by_2.compile()
mul_fcn = c.multiply_by_3_and_add_one.compile()

value = c.value.get_value()
print 'Demonstrate collatz conjecture for initial value of %s' % c.value.get_value()

while value != 1:
    value = div_fcn() if value % 2 == 0 else mul_fcn()
    print value

print 'Note that since the value is attached to the class, it persists if functons are recompiled.'
new_div_fcn = c.divide_by_2.compile()
new_mul_fcn = c.multiply_by_3_and_add_one.compile()
for _ in xrange(6):
    value = new_div_fcn() if value % 2 == 0 else new_mul_fcn()
    print value

Demonstrate collatz conjecture for initial value of 20
10
5
16
8
4
2
1
Note that since the value is attached to the class, it persists if functons are recompiled.
4
2
1
4
2
1


## 3. Debugging

A big advantage of Plato is easier debugging.  There are two ways in which Plato helps you debug.

### 3A: Test values

Theano allows you to add "test-values" to your symbolic variables ([see tutorial](http://deeplearning.net/software/theano/tutorial/debug_faq.html)).  This helps to catch shape-errors at compile-time instead of run-time.  However, it can be a bit of extra work for the programmer.  Fortunately, since Plato compiles your functions on the first pass, it can attach test-values "under the hood".

For example, lets look at a matrix multiplication, where we accidently get the shapes of our matrices wrong.  Since all inputs are given test values, we can easily track down the error - the traceback will lead back to the correct line.  This would not have been possible without test values, because the error would occur in the compiled code, which is no-longer linked to the source code.

In [38]:
import numpy as np
from plato.core import symbolic


@symbolic
def forward_pass(x, w):
    print 'x-shape: %s' % (x.tag.test_value.shape, )
    print 'w-shape: %s' % (w.tag.test_value.shape, )
    return x.dot(w)

f = forward_pass.compile()

try:
    # The following will cause an error (because second argument should have shape (4, 3))
    h = f(np.random.randn(5, 4), np.random.rand(3, 4))  
except ValueError as err:
    # If you do not catch the error, you get a stacktrace which points to the line at fault.
    print '%s: %s' % (err.__class__.__name__, err.message)


x-shape: (5, 4)
w-shape: (3, 4)
ValueError: shapes (5,4) and (3,4) not aligned: 4 (dim 1) != 3 (dim 0)


### 3B: Variable Traces

It can also be useful to print/plot traces of intermediate values.  Ordinarily in theano, this would require setting those variables as outputs, and restructuring code to peek at what would normally be an internal variables.  Plato does a bit of magic which allows you to print/plot/do anything with internal variables.  The following example illustrates this: 

In [41]:
import numpy as np
from plato.core import symbolic
import theano.tensor as tt

class Layer:
    
    def __init__(self, w):
        self.w = theano.shared(w)
        
    @symbolic
    def forward_pass(self, x):
        pre_sigmoid = x.dot(self.w)
        tdbprint(current)
        y = tt.nnet.sigmoid(pre_sigmoid)
        return y
    
n_samples = 5
n_in = 4
n_out = 3
    
layer = Layer(np.random.randn(n_in, n_out))
fwd_fcn = layer.forward_pass.compile()
y = fwd_fcn(np.random.randn(n_samples, n_in))



NameError: global name 'tdbprint' is not defined